<a href="https://colab.research.google.com/github/RifleB-1311/baitap/blob/main/Do_An.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2,os,math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from sklearn.preprocessing import LabelBinarizer
from skimage.transform import resize
from skimage.color import rgb2gray
from skimage import io
from sklearn import preprocessing,metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegressionCV,LogisticRegression
from sklearn.metrics import precision_score,accuracy_score,recall_score,f1_score,precision_recall_curve,roc_auc_score, roc_curve
def read_data(path):
    X_meo=os.listdir(path[0])
    X_cho=os.listdir(path[1])
    y_meo=os.listdir(path[0])
    y_cho=os.listdir(path[1])
    X = []
    y = []
    for i in range(len(X_meo)):
        try:
            image = io.imread(os.path.join(path[0],X_meo[i]))
            image = resize(image,(32,32),anti_aliasing=True)
            gray = rgb2gray(image)
            X.append(list(gray.flatten()))
            y.append(y_meo[i])
        except:
            print("anh meo bi loi",X_meo[i])
    X1 = []
    y1 = []

    for j in range(len(X_cho)):
        try:
            image2 = io.imread(os.path.join(path[1],X_cho[j]))
            image2 = resize(image2,(32,32),anti_aliasing=True)
            gray = rgb2gray(image2)
            X1.append(list(gray.flatten()))
            y1.append(y_cho[j])
        except:
            print("anh cho bi loi",X_cho[j])
    return X,y,X1,y1

def huan_luyen_du_lieu(X,y,X1,y1):
    X_true = np.concatenate((X, X1), axis=0)
    ycat=[0]*len(y)
    ydog=[1]*len(y1)
    y_true = np.concatenate((ycat, ydog), axis=0)
    X=np.array(X_true)
    y=np.array(y_true)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=30)
    return X_train, X_test, y_train, y_test
def y_du_doan(X_train,y_train,X_test,y_test):
    model = LogisticRegressionCV(cv=3, random_state=15).fit(X_test, y_test)
    yhat = model.predict(X_test)
    kNN = KNeighborsClassifier()
    kNN.fit(X_train, y_train)
    yhatKNN = kNN.predict(X_test)
    return  yhat,yhatKNN
def hieu_nang_mo_hinh(y_test,yhatKNN,yhat):
    accuricyKNN=accuracy_score(yhatKNN, y_test)
    precisionKNN=precision_score(y_test, yhatKNN,average='macro')
    recallKNN =recall_score(y_test, yhatKNN, average='macro')
    f1KNN= 2*(precisionKNN*recallKNN)/(precisionKNN+recallKNN)
    accuricy=accuracy_score(yhat, y_test)
    precision=precision_score(y_test, yhat,average='macro')
    recall =recall_score(y_test, yhat, average='macro')
    f1= 2*(precision*recall)/(precision+recall)
    return accuricyKNN,precisionKNN,recallKNN,f1KNN,accuricy,precision,recall,f1
def confunction_matrix(y_test,yhatKNN,yhat):
    confusion_matrix = metrics.confusion_matrix(y_test, yhatKNN)
    TP1=confusion_matrix[0,0]
    FN1=confusion_matrix[0,1]
    FP1=confusion_matrix[1,0]
    TN1=confusion_matrix[1,1]
    precision_KNN=TP1/(TP1+FP1)
    recall_KNN=TP1/(TP1+FN1)
    f1_KNN=2*(precision_KNN*recall_KNN)/(precision_KNN+recall_KNN)
    print("mô hình confunction matrix độ đo presion KNN ",precision_KNN)
    print("mô hình confunction matrix độ đo recall KNN",recall_KNN)
    print("mô hình confunction matrix độ đo f1 KNN",f1_KNN)
    confusion_matrix = metrics.confusion_matrix(y_test, yhat)
    print(confusion_matrix)
    TP=confusion_matrix[0,0]
    FN=confusion_matrix[0,1]
    FP=confusion_matrix[1,0]
    TN=confusion_matrix[1,1]
    precision_logistics=TP/(TP+FP)
    recall_logistics=TP/(TP+FN)
    f1_logistics=2*(precision_logistics*recall_logistics)/(precision_logistics+recall_logistics)
    print("mô hình confunction matrix độ đo presion",precision_logistics)
    print("mô hình confunction matrix độ đo recall",recall_logistics)
    print("mô hình confunction matrix độ đo f1",f1_logistics)
def ve_do_thi(X_train, y_train,X_test,y_test):
    lg_model = LogisticRegression()
    lg_model.fit(X_train, y_train)
    gnb_model = KNeighborsClassifier()
    gnb_model.fit(X_train, y_train)
    lg_probs = lg_model.predict_proba(X_test)[:,1]
    gnb_probs = gnb_model.predict_proba(X_test)[:,1]
    lg_auc = roc_auc_score(y_test, lg_probs)
    gnb_auc = roc_auc_score(y_test, gnb_probs)
    print('Mô hình Logistic Regression - ROC AUC: ', lg_auc)
    print('Mô hình Naive Bayes - ROC AUC: ', gnb_auc)
    lg_fpr, lg_tpr, _ = roc_curve(y_test, lg_probs)
    gnb_fpr, gnb_tpr, _ = roc_curve(y_test, gnb_probs)
    plt.plot(lg_fpr, lg_tpr, marker ='.', label='Logistic')
    plt.plot(gnb_fpr, gnb_tpr, marker='3', label='K_NN')
    plt.legend()
    plt.show()
    lg_model = LogisticRegression()
    lg_model.fit(X_train, y_train)
    gnb_model = KNeighborsClassifier()
    gnb_model.fit(X_train, y_train)
    lg_probs = lg_model.predict_proba(X_test)
    gnb_probs = gnb_model.predict_proba(X_test)
    lg_probs = lg_probs[:,1]
    gnb_probs = gnb_probs[:,1]
    lg_pre, lg_rec, _ = precision_recall_curve(y_test, lg_probs)
    gnb_pre, gnb_rec, _ = precision_recall_curve(y_test, gnb_probs)
    no_model = len(y_test[y_test==1])/len(y_test)
    plt.plot([0,1], [no_model, no_model], linestyle = '--', label='No model')
    plt.plot(lg_rec, lg_pre, marker ='.', label='Logistic')
    plt.plot(gnb_rec, gnb_pre, marker='o', label='K_NN')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.legend()
    plt.show()


def main():
    X, y, X1, y1 = read_data(path=["C:/Users/ADMIN/Downloads/PetImages/Cat",
                                   "C:/Users/ADMIN/Downloads/PetImages/Dog"])
    X_train, X_test, y_train, y_test = huan_luyen_du_lieu(X, y, X1, y1)
    yhat, yhatKNN = y_du_doan(X_train, y_train, X_test, y_test)
    print("y du doan logistic", yhat)
    print("y du doan KNN", yhatKNN)

    accuricyKNN, precisionKNN, recallKNN, f1KNN, accuricy, precision, recall, f1 = hieu_nang_mo_hinh(y_test, yhatKNN,
                                                                                                     yhat)
    print("accuricy cua KNN la", accuricyKNN)
    print("precision cua KNN la", precisionKNN)
    print("recall cua KNN la", recallKNN)
    print("f1 cua KNN la", f1KNN)
    print("accuricy cua logistic la", accuricy)
    print("precision cua logistic la", precision)
    print("recall cua logistic la", recall)
    print("f1 cua logistic la", f1)
    confunction_matrix(y_test, yhatKNN, yhat)
    ve_do_thi(X_train, y_train, X_test, y_test)


if __name__ == "__main__":
    main()